In [ ]:
import sys
import os
import logging
import pandas as pd
import datasets
from pprint import pprint
KEY = '1-CGMLSM'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_CFDATA': f'_Data/3-Data_CFDATA',
    'DATA_EXTERNAL': f'code/external',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

In [ ]:
import os
import datasets 
import pandas as pd 

path = '_Data/z-Data_AIDATA/CgmLsm_Bf24Af2Af2t8H_5Min_3Cohort_ds0p10_SubTD_bf24af2/NameToData'

split_list = os.listdir(path)
print(split_list)

columns = [#'PID', 'ObsDT', 
       'YearOfBirth', #'UserTimeZone', 'UserTimeZoneOffset',
       'MRSegmentID', 'Selected', 'GenderGroup', 'DiseaseTypeGroup',
       # 'Out', 'In', 'Test',
       # 'Valid', 'Train', 'Split', 'Filter', 'TmpFinal', 'SplitSub'
       ]


split_list = ['In-Train',   
              'In-Valid_T1D',  'In-Valid_T2D',
              'In-Test_T1D', 'In-Test_T2D',
              'Out_T1D', 'Out_T2D',
              ]

li = []
for split in split_list:
    split_path = os.path.join(path, split, 'ds_case')
    print(split_path)
    file_list = os.listdir(split_path)
    ds = datasets.Dataset.load_from_disk(split_path)
    df_path = os.path.join(path, split, 'df_case.p')
    df = pd.read_pickle(df_path)
    df = df[columns]
    df['split'] = split
    df['stratum'] = 'None'
    ds_tag = datasets.Dataset.from_pandas(df)
    assert len(ds_tag) == len(ds)
    ds_all = datasets.concatenate_datasets([ds, ds_tag], axis = 1)
    assert len(ds_all) == len(ds_tag)
    li.append(ds_all)

dataset = datasets.concatenate_datasets(li)
dataset = dataset.rename_column('cf.TargetCGM_Bf24H--input_ids', 'CGMValueBf24h--tid')
dataset = dataset.rename_column('cf.TargetCGM_Af2H--input_ids', 'CGMValueAf2h--tid')
dataset = dataset.rename_column('cf.TargetCGM_Af2Hto8H--input_ids', 'CGMValueAf2to8h--tid')
dataset

In [ ]:
data_config = {'CF_to_CFvocab': {}}
CF_to_CFvocab = data_config['CF_to_CFvocab']

CFName = 'HM5MinStep'
interval_delta = pd.Timedelta(minutes=5)
idx2tkn = [pd.Timestamp('2022-01-01 00:00:00') + interval_delta * i for i in range(24 * 12)]
idx2tkn = [f'{i.hour:02d}:{i.minute:02d}' for i in idx2tkn]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab = data_config['CF_to_CFvocab']
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

CFName = 'CGMValue'
idx2tkn = ["PAD", "UNKNOWN", "MASK"] + [f'Other_{i}' for i in range(0, 7)] + [str(i) for i in range(10, 401)]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

print([i for i in CF_to_CFvocab])

In [ ]:
split_list = ['In-Train',   
              'In-Valid_T1D',  'In-Valid_T2D',
              'In-Test_T1D', 'In-Test_T2D',
              'Out_T1D', 'Out_T2D',
              ]

In [ ]:
from recfldtkn.aidata_base.entry import EntryAIData_Builder

OneEntryArgs = {
    # ----------------- Input Part -----------------
    'Split_Part': {
        'SplitMethod': 'SplitFromColumns',
        'Split_to_Selection': {
            'In-Train': {
                'Rules': [
                    ['split', '==', 'In-Train'],
                ],
                'Op': 'and'
            },
            'In-Valid_T1D': {
                'Rules': [
                    ['split', '==', 'In-Valid_T1D'],
                ],
                'Op': 'and'
            },
            'In-Valid_T2D': {
                'Rules': [
                    ['split', '==', 'In-Valid_T2D'],
                ],
                'Op': 'and'
            },

            'In-Test_T1D': {
                'Rules': [
                    ['split', '==', 'In-Test_T1D'],
                ],
                'Op': 'and'
            },
            'In-Test_T2D': {
                'Rules': [
                    ['split', '==', 'In-Test_T2D'],
                ],
                'Op': 'and'
            },

            'Out_T1D': {
                'Rules': [
                    ['split', '==', 'Out_T1D'],
                ],
                'Op': 'and'
            },
            'Out_T2D': {   
                'Rules': [
                    ['split', '==', 'Out_T2D'],
                ],
                'Op': 'and'
            },
        }
    }, # set to empty dict to avoid split
    'Input_Part': {
        'EntryInputMethod': '1TknInStep',
        'CF_list': [
            'CGMValueBf24h',
            'CGMValueAf2h',
           #  'CGMValueAf2to8h',
        ],
        'BeforePeriods': ['Bf24h'],
        'AfterPeriods': ['Af2h'],
        'TimeIndex': True, 
        'InferenceMode': False, # True, # True, # False, # True, 
        'TargetField': 'CGMValue',
        'TargetRange': [40, 400],
        # 'HM': None, 
        'HM': {'start': -24, 'unit': 'h', 'interval': '5m'},
    }, 
    'Output_Part': {
        'EntryOutputMethod': 'CausalLM',
        'selected_columns': ['PID', 'ObsDT', 'input_ids', 'hm_ids', 'labels', 'split', 'stratum'],
        'set_transform': False,
        'num_proc': 12, 
    },
}

entry = EntryAIData_Builder(OneEntryArgs=OneEntryArgs, SPACE=SPACE)
entry.CF_to_CFvocab = CF_to_CFvocab

In [ ]:
# filtered_dataset = dataset.select(range(1000))
filtered_dataset = dataset

Data = {'ds_case': filtered_dataset}
Data = entry.setup_EntryFn_to_Data(Data, CF_to_CFvocab)

In [ ]:
from datasets import DatasetInfo

DataName = 'CgmLsm_WellDoc_ds0p10'
path = os.path.join(SPACE['DATA_HFDATA'], f'{DataName}')

ds_tfm = Data['ds_tfm']
dataset_info = DatasetInfo.from_dict({'config_name': data_config})
ds_tfm.info.update(dataset_info)

ds_tfm

In [ ]:
dataset = ds_tfm
split_to_dataset = entry.split_cf_dataset(dataset)
split_to_dataset.save_to_disk(path)
print(split_to_dataset)